<a href="https://colab.research.google.com/github/Dmitri9149/TensorFlow-PyTorch-basics/blob/master/TensorFlow_Blocks_and_Layesr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import tensorflow as tf

The code is based on the d2l.ai book http://d2l.ai/

In [63]:
### the code is based on d2l.ai book 

In [64]:
### sequence of models , Dense with relu is the first in the chain
net = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256,activation=tf.nn.relu),
  tf.keras.layers.Dense(10)
])

X=tf.random.uniform((2,20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.56546867,  0.01764693,  0.12848157,  0.2398602 ,  0.4043935 ,
         0.4614098 , -0.22573483, -0.01341298, -0.07529312,  0.11078451],
       [ 0.3556044 , -0.13035467,  0.07701491,  0.07666928,  0.18987787,
         0.35594028, -0.03722225,  0.2919003 , -0.11589943,  0.26892406]],
      dtype=float32)>

In [65]:
net.call(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.56546867,  0.01764693,  0.12848157,  0.2398602 ,  0.4043935 ,
         0.4614098 , -0.22573483, -0.01341298, -0.07529312,  0.11078451],
       [ 0.3556044 , -0.13035467,  0.07701491,  0.07666928,  0.18987787,
         0.35594028, -0.03722225,  0.2919003 , -0.11589943,  0.26892406]],
      dtype=float32)>

In [66]:
### Custom Block

In [67]:
class MLP(tf.keras.Model):
  def __init__(self):
    super().__init__()

    self.hidden=tf.keras.layers.Dense(units=256,activation=tf.nn.relu)
    self.out = tf.keras.layers.Dense(units=10)

  def call(self, X):
    return self.out(self.hidden((X)))

In [68]:
net=MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.19470611,  0.21381626,  0.31052095,  0.21637839,  0.14698115,
        -0.0906699 , -0.34867948,  0.35801703, -0.29697287,  0.37223473],
       [ 0.39862064,  0.11056758,  0.55325747,  0.4088604 ,  0.06883898,
        -0.13675705, -0.57990885,  0.30800873, -0.2364105 ,  0.08600396]],
      dtype=float32)>

In [69]:
class MySequential(tf.keras.Model):
  def __init__(self, *args):
    super().__init__()
    self.modules = []
    for block in args:
      self.modules.append(block)

  def call(self,X):
    for module in self.modules:
      X=module(X)
    return(X)


In [70]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.11087985,  0.05098328,  0.08411716,  0.3972369 , -0.2732113 ,
         0.2544626 , -0.3475876 , -0.00500064, -0.21832229,  0.22245055],
       [ 0.3452663 , -0.06318459, -0.08555375,  0.67091405, -0.1395563 ,
         0.1741815 , -0.2779737 ,  0.1991906 , -0.21439165,  0.34139252]],
      dtype=float32)>

Below the Module is constructed where several models are applied in parallel on the same input data and the results are concatenated. It is intended to use the code in multihead attention in Transformers.

In [71]:
#initialize the list of n models which is constructed from the same model
#
def list_of_models(model, n):
  return [model]*n


In [72]:
list_models = [tf.keras.layers.Dense(10)]*3
arg_list = list_of_models(tf.keras.layers.Dense(10),3)

In [73]:
### use *args to supply a list of models of variable length 
class MyParallel(tf.keras.Model):
  def __init__(self, *args):
    super().__init__()
    self.modules = []
    for block in args:
      self.modules.append(block)

  def call(self,X):
    list_res= []
    for module in self.modules:
      list_res.append(module(X))
#      print(len(list_res))
#    print(list_res)
    concat_final= tf.concat(list_res, -1)
    return(concat_final)

In [74]:
net = MyParallel(tf.keras.layers.Dense(10), tf.keras.layers.Dense(10), tf.keras.layers.Dense(10))
X = tf.random.uniform((2,10))
net(X)

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[-0.04144198,  0.13751586, -0.45440757, -0.42069888,  0.32351312,
         0.09995229,  0.52204984, -0.8588798 ,  0.8909292 ,  0.27593744,
         0.00318559,  0.7475995 , -0.5300903 ,  0.2302008 , -0.41739255,
         0.5752885 ,  0.9930589 ,  0.6537688 ,  0.7941842 , -0.70428973,
         0.02429855, -0.19240324,  0.2189645 ,  0.7190653 ,  0.15850393,
         0.08497703,  0.15591794, -0.8616266 ,  0.3332233 , -1.3446494 ],
       [ 0.48957825,  0.82538337, -0.28344512,  0.11973459,  0.6050488 ,
         0.71347564,  0.1660837 , -0.93815887,  0.6253483 ,  0.2953052 ,
        -1.1472697 ,  1.1694889 , -0.6445564 , -0.5370617 , -0.06383955,
         0.87613976,  0.34548354,  0.4736285 ,  0.87099123, -1.7765149 ,
         0.21711344, -0.29982686,  0.80154985,  0.9742272 ,  0.10242325,
         1.2912749 ,  0.8196146 , -0.593518  ,  0.33576167, -0.78474534]],
      dtype=float32)>

In [75]:
### to use the list_models (which is a list) we have to unpack it
net = MyParallel(*list_models)
X = tf.random.uniform((2,10))
net(X)

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[-1.6343818 ,  0.43065274,  1.0249429 ,  1.4949062 ,  0.50572354,
        -0.67563117, -0.53408897, -0.4234969 , -0.5841254 , -0.20707145,
        -1.6343818 ,  0.43065274,  1.0249429 ,  1.4949062 ,  0.50572354,
        -0.67563117, -0.53408897, -0.4234969 , -0.5841254 , -0.20707145,
        -1.6343818 ,  0.43065274,  1.0249429 ,  1.4949062 ,  0.50572354,
        -0.67563117, -0.53408897, -0.4234969 , -0.5841254 , -0.20707145],
       [-1.4047246 ,  0.82266486,  0.6274249 ,  1.2966945 , -0.13676372,
        -0.45226094,  0.32211983, -0.45091963,  0.26938742, -0.07669503,
        -1.4047246 ,  0.82266486,  0.6274249 ,  1.2966945 , -0.13676372,
        -0.45226094,  0.32211983, -0.45091963,  0.26938742, -0.07669503,
        -1.4047246 ,  0.82266486,  0.6274249 ,  1.2966945 , -0.13676372,
        -0.45226094,  0.32211983, -0.45091963,  0.26938742, -0.07669503]],
      dtype=float32)>

In [76]:
net = MyParallel(*arg_list)
X = tf.random.uniform((2,10))
net(X)

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[-0.2117552 , -1.0303696 ,  1.0284584 , -0.32443723,  1.4101036 ,
         0.13429444,  0.2480225 ,  0.09467842,  0.5592507 ,  0.7357806 ,
        -0.2117552 , -1.0303696 ,  1.0284584 , -0.32443723,  1.4101036 ,
         0.13429444,  0.2480225 ,  0.09467842,  0.5592507 ,  0.7357806 ,
        -0.2117552 , -1.0303696 ,  1.0284584 , -0.32443723,  1.4101036 ,
         0.13429444,  0.2480225 ,  0.09467842,  0.5592507 ,  0.7357806 ],
       [-0.21125972, -0.9290842 ,  0.65989983, -0.21173945,  1.0153315 ,
         0.2947011 ,  0.5576079 ,  0.26292172, -0.12913175,  0.7654324 ,
        -0.21125972, -0.9290842 ,  0.65989983, -0.21173945,  1.0153315 ,
         0.2947011 ,  0.5576079 ,  0.26292172, -0.12913175,  0.7654324 ,
        -0.21125972, -0.9290842 ,  0.65989983, -0.21173945,  1.0153315 ,
         0.2947011 ,  0.5576079 ,  0.26292172, -0.12913175,  0.7654324 ]],
      dtype=float32)>

In [77]:
### Nested Models 
class NestedBlocks(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.net=tf.keras.Sequential()
    self.net.add(tf.keras.layers.Dense(20,activation = tf.nn.relu))
    self.net.add(tf.keras.layers.Dense(20,activation = tf.nn.relu))
    self.dense = tf.keras.layers.Dense(20,activation = tf.nn.relu)

  def call(self, inputs):
    return self.dense(self.net(inputs))



In [78]:
net = NestedBlocks()
X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[0.06964082, 0.        , 0.        , 0.        , 0.        ,
        0.57986   , 0.        , 0.08175316, 0.        , 0.        ,
        0.11153333, 0.14937934, 0.        , 0.33071035, 0.13170797,
        0.        , 0.        , 0.        , 0.1404232 , 0.        ],
       [0.32230416, 0.        , 0.        , 0.        , 0.        ,
        0.21086219, 0.        , 0.169721  , 0.        , 0.        ,
        0.28218508, 0.0095907 , 0.        , 0.        , 0.22259459,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>

Parameter access

In [79]:
import numpy as np

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.2878107],
       [0.5349996]], dtype=float32)>

In [80]:
print(net.layers[2].weights)

[<tf.Variable 'dense_50/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[ 0.6473706 ],
       [-0.49180436],
       [-0.9194    ],
       [ 0.45318627]], dtype=float32)>, <tf.Variable 'dense_50/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [81]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_50/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


In [82]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense_49/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.03430647, -0.06631202, -0.63923466, -0.14278638],
       [ 0.40283448, -0.85904443,  0.8333606 ,  0.32834476],
       [ 0.67538065,  0.7536586 , -0.25492835,  0.40667862],
       [ 0.6236997 ,  0.70244664,  0.06181371,  0.4390474 ]],
      dtype=float32)>, <tf.Variable 'dense_49/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[-0.03430647, -0.06631202, -0.63923466, -0.14278638],
       [ 0.40283448, -0.85904443,  0.8333606 ,  0.32834476],
       [ 0.67538065,  0.7536586 , -0.25492835,  0.40667862],
       [ 0.6236997 ,  0.70244664,  0.06181371,  0.4390474 ]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[ 0.6473706 ],
       [-0.49180436],
       [-0.9194    ],
       [ 0.45318627]], dtype=float32), array([0.], dtype=float32)]


In [83]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

In [84]:
### collecting parameters from nested blocks
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.02913161],
       [-0.05884419]], dtype=float32)>

In [85]:
print(rgnet.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_15 (Sequential)   (2, 4)                    80        
_________________________________________________________________
dense_55 (Dense)             (2, 1)                    5         
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [86]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'sequential_15/block-1/dense_52/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

In [87]:
net_prl = MyParallel(tf.keras.layers.Dense(4), tf.keras.layers.Dense(4), tf.keras.layers.Dense(4))
X = tf.random.uniform((1,4))
net_prl(X)

<tf.Tensor: shape=(1, 12), dtype=float32, numpy=
array([[ 0.05302444,  0.20611267, -0.32832617,  0.2658228 , -0.31834635,
         0.16720435,  0.15011795,  0.40936515,  0.51037914,  0.04175417,
        -0.6982855 , -0.3057128 ]], dtype=float32)>

In [88]:
print(net_prl.summary())

Model: "my_parallel_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_56 (Dense)             multiple                  20        
_________________________________________________________________
dense_57 (Dense)             multiple                  20        
_________________________________________________________________
dense_58 (Dense)             multiple                  20        
Total params: 60
Trainable params: 60
Non-trainable params: 0
_________________________________________________________________
None


In [89]:
net = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(
      4, activation = tf.nn.relu, 
      kernel_initializer = tf.random_normal_initializer(mean=0, stddev=0.01),
      bias_initializer=tf.zeros_initializer()
  ),
  tf.keras.layers.Dense(1)                                  
])
net(X)
net.weights

[<tf.Variable 'dense_59/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-2.0386739e-02,  3.1833693e-03, -4.8929360e-04,  9.0800244e-03],
        [ 1.2955199e-03,  4.4713989e-03, -1.7108226e-02, -9.6423103e-04],
        [ 6.0304701e-03,  3.4701421e-03,  4.8880634e-04,  3.0424586e-03],
        [ 7.2907380e-05,  1.9009771e-03, -3.6511968e-03, -7.5835777e-03]],
       dtype=float32)>,
 <tf.Variable 'dense_59/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_60/kernel:0' shape=(4, 1) dtype=float32, numpy=
 array([[ 0.5442349 ],
        [-0.1171425 ],
        [ 0.7243327 ],
        [-0.65060705]], dtype=float32)>,
 <tf.Variable 'dense_60/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [90]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_61/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_61/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [91]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(1)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_63/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 0.40403908,  0.20973986, -0.40347582,  0.84569365],
       [-0.41068926,  0.4895348 ,  0.45490128, -0.36813632],
       [ 0.43737882,  0.44880754,  0.44345385, -0.2388019 ],
       [ 0.4401378 , -0.18698454, -0.11230856, -0.8334196 ]],
      dtype=float32)>
<tf.Variable 'dense_64/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>


In [92]:
class MyInit(tf.keras.initializers.Initializer):
  def __init__(self,shape, dtype=None):
    data = tf.random.uniform(shape, -10,10,dtype=dtype)
    factor=(tf.abs.data >=5)
    factor=tf.cast(factor, tf.float32)
    return factor*data

net(X)
net.layers[1].weights

[<tf.Variable 'dense_63/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.40403908,  0.20973986, -0.40347582,  0.84569365],
        [-0.41068926,  0.4895348 ,  0.45490128, -0.36813632],
        [ 0.43737882,  0.44880754,  0.44345385, -0.2388019 ],
        [ 0.4401378 , -0.18698454, -0.11230856, -0.8334196 ]],
       dtype=float32)>,
 <tf.Variable 'dense_63/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

In [93]:
### assign parameters
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_63/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.        ,  1.2097399 ,  0.5965242 ,  1.8456936 ],
       [ 0.58931077,  1.4895349 ,  1.4549012 ,  0.6318637 ],
       [ 1.4373789 ,  1.4488075 ,  1.4434538 ,  0.7611981 ],
       [ 1.4401379 ,  0.81301546,  0.88769144,  0.16658038]],
      dtype=float32)>

In [94]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

True


In [95]:
### Deffered Initialization
net = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, activation=tf.nn.relu),
  tf.keras.layers.Dense(10)                                  
])

In [96]:
### layers are empty
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

In [97]:
### this code gives error because net the weights are not initilized yet
# net.get_weights()

In [98]:
X = tf.random.uniform((2,20))
net(X)
[w.shape for w in net.get_weights()]

[(20, 256), (256,), (256, 10), (10,)]

In [99]:
### Custom Layers
class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        return inputs - tf.reduce_mean(inputs)

In [100]:
layer = CenteredLayer()
layer(tf.constant([1,2,3,4]))

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([-1,  0,  1,  2], dtype=int32)>

In [101]:
net = tf.keras.Sequential([tf.keras.layers.Dense(10), CenteredLayer()])

In [102]:
Y = net(tf.random.uniform((4,20)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=1.1920929e-08>

In [103]:
### Linear layer from scratch
class MyDense(tf.keras.Model):
    def __init__(self, out_units):
        super().__init__()
        self.out_units = out_units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.out_units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.out_units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [104]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[ 0.00663937, -0.03279245, -0.05857573],
        [-0.08733882,  0.00432103,  0.07845158],
        [-0.00502899,  0.03129234, -0.04915142],
        [ 0.00672116, -0.04013789, -0.06891046],
        [-0.12798248,  0.02999985, -0.01782634]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [105]:
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.        , 0.        , 0.        ],
       [0.        , 0.01689125, 0.        ]], dtype=float32)>

In [106]:
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.],
       [0.]], dtype=float32)>